<a href="https://colab.research.google.com/github/RafaelGallo/LLM/blob/main/LLM__Gemma_cluster_sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando pacotes
!pip install -U keras-nlp
!pip install -U keras

In [ ]:
!pip install tensorflow-gpu

In [ ]:
# Tipo GPU
!nvidia-smi

In [ ]:
# Versão GPU
!nvcc --version

In [ ]:
# Versão keras tensorflow
import tensorflow as tf
print("Versão tensorflow:", tf.__version__)

import keras
print("Versão keras:", keras.__version__)

In [ ]:
# Importando biblioteca
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

In [ ]:
## Base dados

# Dados de exemplo
sentences = [
             "Eu amo este filme, é ótimo!",
             "Que livro chato, não consegui terminar de ler",
             "Esta comida é deliciosa",
             "O serviço neste restaurante é terrível",
             "Que bela vista!",
             "Este produto é de péssima qualidade"
             ]

# Rótulos de exemplo (sentimentos positivos ou negativos)
labels = np.array([1, 0, 1, 0, 1, 0])

In [ ]:
# Pré-processamento dos dados
vectorizer = TfidfVectorizer()

# Treinamento modelo
X = vectorizer.fit_transform(sentences).toarray()

In [ ]:
# Adicionando uma dimensão de tempo para os dados de entrada
X = np.expand_dims(X, axis=1)

In [ ]:
# Modelo LLM
class LLM_Clustering:
    def __init__(self, input_dim, hidden_units, num_clusters):
        self.input_dim = input_dim
        self.hidden_units = hidden_units
        self.num_clusters = num_clusters

        # Criação do modelo LSTM
        inputs = Input(shape=(None, input_dim))
        lstm_layer = LSTM(hidden_units)
        lstm_output = lstm_layer(inputs)
        output_layer = Dense(num_clusters, activation='softmax')
        outputs = output_layer(lstm_output)

        # Definição do modelo
        self.model = Model(inputs=inputs, outputs=outputs)

        # Compilação do modelo
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, X, y, batch_size, epochs):
        self.model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    def predict(self, X):
        return self.model.predict(X)

In [ ]:
# Método do cotovelo para encontrar o número ideal de clusters
inertia_values = []
max_clusters = min(len(sentences) - 1, 9)  # Limitar o número máximo de clusters ao número de amostras menos 1
for num_clusters in range(2, max_clusters + 1):  # Começar a partir de 2 clusters
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    inertia_values.append(kmeans.inertia_)

# Plotar o gráfico de cotovelo
plt.plot(range(2, max_clusters + 1), inertia_values, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.show()


In [ ]:
# Treinamento do modelo LLM para classificação de sentimentos
input_dim = X.shape[1]
hidden_units = 64

# Número de clusters
num_clusters = 2

In [ ]:
# Modelo LLM
llm_model = LLM_Clustering(X.shape[2], hidden_units, num_clusters)

In [ ]:
# Treinamento modelo
model_llm = llm_model.train(X,
                            tf.keras.utils.to_categorical(labels),
                            batch_size=32,
                            epochs=20)

In [ ]:
# Obtenção dos embeddings usando o modelo LLM
embeddings = llm_model.predict(X)

In [ ]:
# Clusterização usando K-Means nos embeddings do modelo LSTM
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)

In [ ]:
# Imprimir os rótulos dos clusters e o número de clusters únicos
print("Rótulos dos Clusters:", kmeans.labels_)
print("Número de Clusters Únicos:", num_clusters)

In [ ]:
# Avaliação da qualidade da clusterização
num_clusters = len(np.unique(kmeans.labels_))
if num_clusters > 1:
    silhouette_avg = silhouette_score(embeddings, kmeans.labels_)
    print("Silhouette Score:", silhouette_avg)
else:
    print("Não é possível calcular o escore de silhueta com apenas um cluster.")


In [ ]:
# Análise dos resultados
for i in range(num_clusters):
    cluster_sentences = np.array(sentences)[kmeans.labels_ == i]
    print("Cluster", i, ":", cluster_sentences)

In [ ]:
# Mapeamento dos rótulos de cluster para sentimentos
sentiment_labels = ['Positivo', 'Negativo', 'Neutro']  # Defina os rótulos de acordo com sua análise de sentimentos

# Número ideal de clusters (definido manualmente ou usando o método do cotovelo)
num_clusters = 3

# Clusterização usando K-Means com o número ideal de clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)
cluster_labels = kmeans.labels_

# Plotar os clusters
plt.figure(figsize=(8, 6))
for cluster in range(num_clusters):
    cluster_sentences = np.array(sentences)[cluster_labels == cluster]
    plt.scatter(embeddings[cluster_labels == cluster, 0], embeddings[cluster_labels == cluster, 1], label=sentiment_labels[cluster])
plt.title('Clusterização modelo LLM - Gemma')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.legend()
plt.show()
